# 1: Initialisation

In [2]:
# import necessary modules for the rest of the code to run without errors
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import joblib

# model related modules
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import (StandardScaler, MinMaxScaler) # after testing we found Standard and MinMax worked best for our models
import statsmodels.api as sm
import statsmodels.tools
from sklearn.linear_model import LassoCV

# 2: Optimised Model

## 2.1: Feature Engineering

In [3]:
# read the data from the provided csv
df = pd.read_csv("Life Expectancy Data.csv")

#### Data-independent feature engineering
* **OHE Region**
* **OHE Country** `(later removed)`
* Calculate new field: **log of GDP**
* Calculate new field: **average immunisation**


In [4]:
# this function contains the feature engineering that will not result in data leakage
def pre_split_feature_eng(df):
    df = df.copy()
    # df = pd.get_dummies(df, columns = ['Country'], drop_first = True, prefix = 'Country', dtype = int) # we decided not to use country
    df = pd.get_dummies(df, columns = ['Region'], drop_first = True, prefix = 'Region', dtype = int) # OHE for region
    df['log_GDP'] = np.log(df['GDP_per_capita']) # create a new field for log(GDP)
    df['immunisation_avg'] = (df['Polio'] + df['Diphtheria'] + df['Hepatitis_B']) / 3 # average immunisation calculation
    return df

In [5]:
OHE_df = pre_split_feature_eng(df) # apply above feature engineering to the entire dataset

In [6]:
feature_cols = list(OHE_df.columns) # split the features from the target
feature_cols.remove('Life_expectancy')

In [7]:
# X represents our features, y represents the target
X = OHE_df[feature_cols]
y = OHE_df['Life_expectancy']

In [8]:
# split using the normal test size of 20% and a constant state for consistency across models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#### Data-dependent feature engineering
* Scaling `(E.g. Standard scaling)`

#### Model features:
* Year
* Region
* Under_five_deaths
* Adult_mortality
* BMI
* Incidents_HIV
* GDP_per_capita
* Schooling
* Economy_status_Developing

#### Selected using a combination of:
* VIF
* Stepwise
* Lasso
* Trial and error

In [9]:
# columns used by our model that require scaling include:
scale_cols = ['Schooling', 'Adult_mortality', 'Under_five_deaths', 'GDP_per_capita', 'Year', 'log_GDP', 'BMI', 'Incidents_HIV']

# we tested several scalers and decided on standardscaler
scaler = StandardScaler()
scaler.fit(X_train[scale_cols]) # we fit our scaler on the TRAINING set, and not the testing data
joblib.dump(scaler, 'opti_scaler.pkl') # dump our scaler to a pickle file to be used by our function notebook

def feature_eng(df, scaler, scale_cols): # this function applies the scaler values to a specified set of columns in a dataframe
    df = df.copy()
    df[scale_cols] = scaler.transform(df[scale_cols])
    df = sm.add_constant(df) # each row requires a constant value
    return df

In [10]:
X_train_fe = feature_eng(X_train, scaler, scale_cols) # apply the standard scaler to the train set

## 2.2: Fitting the Model

In [11]:
# dropping 'Economy_status_Developing' at index: 17
# dropping 'Polio' at index: 8
# dropping 'Infant_deaths' at index: 1
# dropping 'Diphtheria' at index: 7
# dropping 'Schooling' at index: 12
# dropping 'Hepatitis_B' at index: 4
# dropping 'Thinness_ten_nineteen_years' at index: 9
# dropping 'Adult_mortality' at index: 2
# dropping 'BMI' at index: 4
# dropping 'Measles' at index: 3
# Cond 17.9 RMSE 2.2

# dropping 'Economy_status_Developing' at index: 17
# dropping 'Polio' at index: 8
# dropping 'Infant_deaths' at index: 1
# dropping 'Diphtheria' at index: 7
# dropping 'Schooling' at index: 12
# dropping 'Hepatitis_B' at index: 4
# Cond 39.6 RMSE 1.2

#### Model Results:
* High R^2 of `0.984`
* Low relative AIC and BIC
* Low condition number of `25.2`

In [12]:
# selected features as a result of trial and error, VIF, Lasso and Stepwise testing
feature_cols = ['const', 'Schooling', 'Adult_mortality', 'Under_five_deaths', 'Economy_status_Developing', 'Region_Central America and Caribbean', 'Region_South America', 'GDP_per_capita', 'Region_Oceania', 'Region_European Union', 'Year', 'log_GDP', 'BMI', 'Incidents_HIV', 'Region_Rest of Europe', 'Region_North America']

lin_reg = sm.OLS(y_train, X_train_fe[feature_cols]) # creating the linear regression model
results = lin_reg.fit() # traininf the model
joblib.dump(results, 'opti_model.pkl') # saving the model to a file for our function notebook
results.summary() # viewing results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     9259.
Date:                Wed, 16 Jul 2025   Prob (F-statistic):               0.00
Time:                        08:45:58   Log-Likelihood:                -3674.0
No. Observations:                2291   AIC:                             7380.
Df Residuals:                    2275   BIC:                             7472.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   70.6152      0.159    444.048      0.000      70.303      70.927
Schooling                                0.3440      0.054      6.392      0.000       0.238       0.449
Adult_mortality                         -5.4056      0.071    -76.183      0.000      -5.545      -5.266
Under_five_deaths                       -3.4429      0.065    -52.861      0.000      -3.571      -3.315
Economy_status_Developing               -2.5135      0.164    -15.357      0.000      -2.834      -2.193
Region_Central America and Caribbean     1.8301      0.097     18.905      0.000       1.640       2.020
Region_South America                     1.5339      0.109     14.012      0.000       1.319       1.749
GDP_per_capita                           0.1304      0.056      2.345      0.019       0.021       0.239
Region_Oceania                          -0.8664      0.125     -6.917      0.000      -1.112      -0.621
Region_European Union                   -0.7054      0.157     -4.479      0.000      -1.014      -0.397
Year                                     0.1566      0.026      5.945      0.000       0.105       0.208
log_GDP                                  0.5035      0.078      6.450      0.000       0.350       0.657
BMI                                     -0.3411      0.043     -7.911      0.000      -0.426      -0.257
Incidents_HIV                            0.1779      0.044      4.039      0.000       0.092       0.264
Region_Rest of Europe                    0.3850      0.113      3.409      0.001       0.164       0.606
Region_North America                     0.7020      0.215      3.264      0.001       0.280       1.124
==============================================================================
Omnibus:                       33.967   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.839
Skew:                           0.165   Prob(JB):                     2.48e-11
Kurtosis:                       3.635   Cond. No.                         25.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2.3 Testing Optimised Prediction

In [13]:
y_pred_train = results.predict(X_train_fe[feature_cols]) # use the model to make predictions on training feature data

rmse_train = statsmodels.tools.eval_measures.rmse(y_train, y_pred_train) # calculate rmse using predicted and actual target values

print(f'Train RMSE: {round(rmse_train,3)}') 

Train RMSE: 1.203


In [14]:
X_test_fe = feature_eng(X_test, scaler, scale_cols) # feature engineer and scale the testing features using the TRAINING scale
X_test_fe = X_test_fe[feature_cols] # select the features

In [16]:
y_test_pred = results.predict(X_test_fe) # predict target using test features
rmse = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred) # calculate rmse of predicted and actual test target values
print(f'Test RMSE: {round(rmse,3)}')

Test RMSE: 1.226


#### Possible indication of overfitting:
* `0.02` is relatively small
* Feature selection was performed using `Lasso`

# 3: Feature Selection Metrics

> Trial and error: We repeatedly adjusted the feature selection thresholds and tested the results until we found the model with the best performance.

## 3.1: Calculate VIF (Variance Inflation Factor)

In [16]:
def calculate_vif(X, thresh = 10):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        # this bit uses list comprehension to gather all the VIF values of the different variables
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]
        
        maxloc = vif.index(max(vif)) # getting the index of the highest VIF value
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc] # we delete the highest VIF value on condition that it's higher than the threshold
            dropped = True # if we deleted anything, we set the 'dropped' value to True to stay in the while loop

    print('Remaining variables:')
    print(X.columns[variables]) # finally, we print the variables that are still in our set
    return X.iloc[:, variables] # and return our X cut down to the remaining variables

In [17]:
VIF_variables = calculate_vif(X_train_fe[['Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'Region_Asia',
       'Region_Central America and Caribbean', 'Region_European Union',
       'Region_Middle East', 'Region_North America', 'Region_Oceania',
       'Region_Rest of Europe', 'Region_South America', 'log_GDP', 'immunisation_avg']])

# Error shows 'RuntimeWarning: divide by zero encountered in scalar divide vif = 1. / (1. - r_squared_i)' this is because
# the R^2 for some features is 1 resulting in 1 / 0, this causes the error since VIF is essentially infinite.
# this happens when the variance in the target for a specific feature is fully explainable by a combination of other features
# high levels of multicollinearity

dropping 'Hepatitis_B' at index: 5


C:\Users\legom\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


dropping 'Economy_status_Developing' at index: 16
dropping 'immunisation_avg' at index: 25
dropping 'Polio' at index: 7
dropping 'Infant_deaths' at index: 1
dropping 'Diphtheria' at index: 6
dropping 'Thinness_ten_nineteen_years' at index: 9
dropping 'log_GDP' at index: 20
Remaining variables:
Index(['Year', 'Under_five_deaths', 'Adult_mortality', 'Alcohol_consumption',
       'Measles', 'BMI', 'Incidents_HIV', 'GDP_per_capita', 'Population_mln',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Region_Asia', 'Region_Central America and Caribbean',
       'Region_European Union', 'Region_Middle East', 'Region_North America',
       'Region_Oceania', 'Region_Rest of Europe', 'Region_South America'],
      dtype='object')


## 3.2: Stepwise

In [18]:
def stepwise_selection(X, y, threshold_in = 0.01, threshold_out = 0.3, verbose = True):
    # The function is checking for p-values (whether features are statistically significant) - lower is better
    included = [] # this is going to be the list of features we keep
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index = excluded, dtype = 'float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        # we add the feature with the lowest (best) p-value under the threshold to our 'included' list
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval)) # specifying the verbose text


        # backward step: removing features if new features added to the list make them statistically insignificant
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        # if the p-value exceeds the upper threshold, the feature will be dropped from the 'included' list
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [19]:
result = stepwise_selection(X_train_fe[['Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'Region_Asia',
       'Region_Central America and Caribbean', 'Region_European Union',
       'Region_Middle East', 'Region_North America', 'Region_Oceania',
       'Region_Rest of Europe', 'Region_South America', 'log_GDP', 'immunisation_avg']], y_train)

print('resulting features:')
print(result)

Add  log_GDP                        with p-value 0.0
Add  Under_five_deaths              with p-value 0.0
Add  Adult_mortality                with p-value 0.0
Add  Economy_status_Developing      with p-value 7.3162e-60
Add  Economy_status_Developed       with p-value 0.0
Add  Region_Central America and Caribbean with p-value 1.73497e-43
Add  Region_South America           with p-value 8.86468e-33
Add  Region_Oceania                 with p-value 2.5627e-26
Add  Region_European Union          with p-value 3.14928e-24
Add  Infant_deaths                  with p-value 1.63493e-14
Add  Schooling                      with p-value 6.70252e-08
Add  BMI                            with p-value 6.56748e-13
Add  Year                           with p-value 6.16713e-08
Add  Hepatitis_B                    with p-value 0.000648365
Add  Incidents_HIV                  with p-value 0.000316316
Add  GDP_per_capita                 with p-value 0.00722397
resulting features:
['log_GDP', 'Under_five_deaths', 

## 3.3: Lasso

#### Key findings of lasso:
* Region coefficients all shrank, in most cases to less than half of their OLS values
* GDP-related coefficients all increased

> This indicates that the model currently relies too heavily on regional labels to predict life expectancy. By using lasso as a form of regularisation, we can reduce this memorisation of regions, resulting in a model that can generalise better with new data.

In [20]:
# lasso is another feature selection method which takes into account the VIF and p-values for features. The result of this are the features used in our optimised model
lasso = LassoCV(cv=5)
lasso.fit(X_train_fe[['Schooling', 'Adult_mortality', 'Under_five_deaths', 'Economy_status_Developing', 'Region_Central America and Caribbean', 'Region_South America', 'GDP_per_capita', 'Region_Oceania', 'Region_European Union', 'Year', 'log_GDP', 'BMI', 'Incidents_HIV', 'Region_Rest of Europe', 'Region_North America']], y_train)
print(f'Selected features: {X_train_fe[['Schooling', 'Adult_mortality', 'Under_five_deaths', 'Economy_status_Developing', 'Region_Central America and Caribbean', 'Region_South America', 'GDP_per_capita', 'Region_Oceania', 'Region_European Union', 'Year', 'log_GDP', 'BMI', 'Incidents_HIV', 'Region_Rest of Europe', 'Region_North America']].columns[lasso.coef_ != 0].tolist()}')


mask = lasso.coef_ != 0

# Get the selected feature names
selected_features = X_train_fe[['Schooling', 'Adult_mortality', 'Under_five_deaths', 'Economy_status_Developing', 'Region_Central America and Caribbean', 'Region_South America', 'GDP_per_capita', 'Region_Oceania', 'Region_European Union', 'Year', 'log_GDP', 'BMI', 'Incidents_HIV', 'Region_Rest of Europe', 'Region_North America']].columns[mask]

# Get the corresponding non-zero coefficients
selected_coefs = lasso.coef_[mask]

# Pair them up
for name, coef in zip(selected_features, selected_coefs):
    print(f"{name}: {coef}")

Selected features: ['Schooling', 'Adult_mortality', 'Under_five_deaths', 'Economy_status_Developing', 'Region_Central America and Caribbean', 'Region_South America', 'GDP_per_capita', 'Region_Oceania', 'Region_European Union', 'Year', 'log_GDP', 'BMI', 'Incidents_HIV', 'Region_Rest of Europe', 'Region_North America']
Schooling: 0.3725079113332776
Adult_mortality: -5.3505140407294
Under_five_deaths: -3.4461360447527514
Economy_status_Developing: -2.0755489771301368
Region_Central America and Caribbean: 1.6311290835455194
Region_South America: 1.3044469873876074
GDP_per_capita: 0.17800774768968455
Region_Oceania: -0.763701903572521
Region_European Union: -0.3938499071842748
Year: 0.13724523560099414
log_GDP: 0.5187940902279401
BMI: -0.3053088069437367
Incidents_HIV: 0.12479901736999326
Region_Rest of Europe: 0.23857402022518936
Region_North America: 0.27255463085942655


# 4: Limited Model

## 4.1: Feature Selection

In [132]:
df = pd.read_csv("Life Expectancy Data.csv")         # re-initialise the dataframe

#### Data-independent feature engineering
* OHE Region

#### Model features:
* Year
* Region
* Under_five_deaths
* Adult_mortality
* GDP_per_capita
* Economy_status_Developing

#### Selected using:
* **Ethical considerations**:
    * Use the least metrics possible to train a model still worth using
    * We wanted to avoid using `medical data/records` as much as possible
    * `BMI` and `Incidents_HIV` removed relating to medical concerns
    * `Schooling` removed for socioeconomic reasons that may be defamatory

In [133]:
df = pd.get_dummies(df, columns = ['Region'], drop_first = True, prefix = 'Region', dtype = int) # OHE the region column

feature_cols = [ 'Year',
                 'Region_Asia',
                 'Region_Central America and Caribbean',
                 'Region_European Union',
                 'Region_Middle East',
                 'Region_North America',
                 'Region_Oceania',
                 'Region_Rest of Europe',
                 'Region_South America',
                 'Under_five_deaths',
                 'Adult_mortality',
                 'GDP_per_capita',
                 'Economy_status_Developing']

In [134]:
# X represents our features, y represents our target
X = df[feature_cols]
X = sm.add_constant(X) # don't forget to add the constant
y = df.Life_expectancy

## 4.2 Redo the Test-Train Split

In [101]:
# split using the same split and state as before
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## 4.3: Feature Engineering

#### Data-dependent feature engineering
* Scaling `(E.g. MinMax scaling)`



In [102]:
# below are the features used for our limited model
scale_cols = [ 'Year', 'Under_five_deaths', 'Adult_mortality', 'GDP_per_capita', 'Economy_status_Developing']
scaler = MinMaxScaler() # minmax scaler provided the best results for this model
scaler.fit(X_train[scale_cols]) # fit the scaler values on the TRAINING data
joblib.dump(scaler, 'limited_scaler.pkl') # saved the scaler to a file for our function notebook

def feature_eng(df, scaler, scale_cols):
    df = df.copy()
    df[scale_cols] = scaler.transform(df[scale_cols])
    df = sm.add_constant(df)
    return df

In [103]:
X_train = feature_eng(X_train, scaler, scale_cols) # apply the training scaler to the train features
X_test = feature_eng(X_test, scaler, scale_cols) # apply the testing scaler to the test features

## 4.4: Fitting the Model

#### Model Results:
* High R^2 of 0.983
* Low relative AIC and BIC
* Low condition number of 21.9

In [104]:
lin_reg = sm.OLS(y_train, X_train) # initialising the model object – remember, for sm it's y first and then X
results = lin_reg.fit() # creating an object for the fitted model
joblib.dump(results, 'limited_model.pkl')
y_pred = results.predict(X_train) # adding our predictions back to the table
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 1.014e+04
Date:                Mon, 14 Jul 2025   Prob (F-statistic):               0.00
Time:                        14:56:11   Log-Likelihood:                -3734.2
No. Observations:                2291   AIC:                             7496.
Df Residuals:                    2277   BIC:                             7577.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   80.1342      0.210    382.080      0.000      79.723      80.545
Year                                     0.4970      0.088      5.670      0.000       0.325       0.669
Region_Asia                              0.3626      0.101      3.582      0.000       0.164       0.561
Region_Central America and Caribbean     1.9026      0.118     16.119      0.000       1.671       2.134
Region_European Union                   -0.6891      0.174     -3.962      0.000      -1.030      -0.348
Region_Middle East                       0.0992      0.133      0.746      0.456      -0.161       0.360
Region_North America                     0.6788      0.228      2.978      0.003       0.232       1.126
Region_Oceania                          -1.1276      0.132     -8.521      0.000      -1.387      -0.868
Region_Rest of Europe                    0.4962      0.130      3.820      0.000       0.241       0.751
Region_South America                     1.7102      0.130     13.136      0.000       1.455       1.966
Under_five_deaths                      -18.8613      0.248    -75.956      0.000     -19.348     -18.374
Adult_mortality                        -28.6620      0.272   -105.212      0.000     -29.196     -28.128
GDP_per_capita                           3.0448      0.266     11.465      0.000       2.524       3.566
Economy_status_Developing               -3.0186      0.161    -18.708      0.000      -3.335      -2.702
==============================================================================
Omnibus:                       44.454   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               62.147
Skew:                           0.222   Prob(JB):                     3.20e-14
Kurtosis:                       3.673   Cond. No.                         21.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 4.5: Testing Limited Model

In [106]:
print(f'Train RMSE: {round(statsmodels.tools.eval_measures.rmse(y_train, y_pred),3)}') # calculate the rmse from predicted and actual target values

Train RMSE: 1.235


In [108]:
y_test_pred = results.predict(X_test) # predict for the test data
rmse = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred) # calculate the rmse from predicted and actual target values of the test data
print(f'Test RMSE: {round(rmse,3)}', )

Test RMSE: 1.247
